In [ ]:
!git clone https://github.com/xieyongliang/stable-diffusion-webui.git

In [69]:
import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name
images_s3uri = 's3://{0}/dreambooth/images/'.format(bucket)

In [70]:
imgs="https://d1xkebsgyt7kzd.cloudfront.net/R_1.jpg,https://d1xkebsgyt7kzd.cloudfront.net/R_2.jpg,https://d1xkebsgyt7kzd.cloudfront.net/R_3.jpg,https://d1xkebsgyt7kzd.cloudfront.net/R_4.jpg,https://d1xkebsgyt7kzd.cloudfront.net/R_5.jpg"
for image in imgs.split(","):
    !wget $image
!mv ./*.jpg ./images/
!aws s3 cp images $images_s3uri --recursive    

--2023-02-27 09:27:26--  https://d1xkebsgyt7kzd.cloudfront.net/R_1.jpg
Resolving d1xkebsgyt7kzd.cloudfront.net (d1xkebsgyt7kzd.cloudfront.net)... 13.33.100.75, 13.33.100.45, 13.33.100.108, ...
Connecting to d1xkebsgyt7kzd.cloudfront.net (d1xkebsgyt7kzd.cloudfront.net)|13.33.100.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38281 (37K) [image/jpeg]
Saving to: ‘R_1.jpg’

R_1.jpg             100%[===================>]  37.38K  --.-KB/s    in 0s      

2023-02-27 09:27:26 (306 MB/s) - ‘R_1.jpg’ saved [38281/38281]

--2023-02-27 09:27:27--  https://d1xkebsgyt7kzd.cloudfront.net/R_2.jpg
Resolving d1xkebsgyt7kzd.cloudfront.net (d1xkebsgyt7kzd.cloudfront.net)... 13.33.100.121, 13.33.100.108, 13.33.100.45, ...
Connecting to d1xkebsgyt7kzd.cloudfront.net (d1xkebsgyt7kzd.cloudfront.net)|13.33.100.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33919 (33K) [image/jpeg]
Saving to: ‘R_2.jpg’

R_2.jpg             100%[===================>] 

In [4]:
"""
1:model_name: str = "",
2:adam_beta1: float = 0.9,
3:adam_beta2: float = 0.999,
4:adam_epsilon: float = 1e-8,
5:adam_weight_decay: float = 0.01,
6:attention: str = "default",
7:center_crop: bool = True,
8:concepts_path: str = "",
9:custom_model_name: str = "",
10:epoch_pause_frequency: int = 0,
11:epoch_pause_time: int = 0,
12:gradient_accumulation_steps: int = 1,
13:gradient_checkpointing: bool = True,
14:half_model: bool = False,
15:has_ema: bool = False,
16:hflip: bool = False,
17:learning_rate: float = 0.00000172,
18:lora_learning_rate: float = 1e-4,
19:lora_txt_learning_rate: float = 5e-5,
20:lr_scheduler: str = 'constant',
21:lr_warmup_steps: int = 0,
22:max_token_length: int = 75,
23:max_train_steps: int = 1000,
24:mixed_precision: str = "fp16",
25:model_path: str = "",
26:not_cache_latents=False,
27:num_train_epochs: int = 1,
28:pad_tokens: bool = True,
29:pretrained_vae_name_or_path: str = "",
30:prior_loss_weight: float = 1.0,
31:resolution: int = 512,
32:revision: int = 0,
33:sample_batch_size: int = 1,
34:save_class_txt: bool = False,
35:save_embedding_every: int = 500,
36:save_preview_every: int = 500,
37:save_use_global_counts: bool = False,
38:save_use_epochs: bool = False,
39:scale_lr: bool = False,
40:scheduler: str = "ddim",
41:src: str = "",
42:shuffle_tags: bool = False,
43:train_batch_size: int = 1,
44:train_text_encoder: bool = True,
45:use_8bit_adam: bool = True,
46:use_concepts: bool = False,
47:use_cpu: bool = False,
48:use_ema: bool = True,
49:use_lora: bool = False,
50:v2: bool = False,
51:c1_class_data_dir: str = "",
52:c1_class_guidance_scale: float = 7.5,
53:c1_class_infer_steps: int = 60,
54:c1_class_negative_prompt: str = "",
55:c1_class_prompt: str = "",
56:c1_class_token: str = "",
57:c1_instance_data_dir: str = "",
58:c1_instance_prompt: str = "",
59:c1_instance_token: str = "",
60:c1_max_steps: int = -1,
61:c1_n_save_sample: int = 1,
62:c1_num_class_images: int = 0,
63:c1_sample_seed: int = -1,
64:c1_save_guidance_scale: float = 7.5,
65:c1_save_infer_steps: int = 60,
66:c1_save_sample_negative_prompt: str = "",
67:c1_save_sample_prompt: str = "",
68:c1_save_sample_template: str = "",
69:c2_class_data_dir: str = "",
70:c2_class_guidance_scale: float = 7.5,
71:c2_class_infer_steps: int = 60,
72:c2_class_negative_prompt: str = "",
73:c2_class_prompt: str = "",
74:c2_class_token: str = "",
75:c2_instance_data_dir: str = "",
76:c2_instance_prompt: str = "",
77:c2_instance_token: str = "",
78:c2_max_steps: int = -1,
79:c2_n_save_sample: int = 1,
80:c2_num_class_images: int = 0,
81:c2_sample_seed: int = -1,
82:c2_save_guidance_scale: float = 7.5,
83:c2_save_infer_steps: int = 60,
84:c2_save_sample_negative_prompt: str = "",
85:c2_save_sample_prompt: str = "",
86:c2_save_sample_template: str = "",
87:c3_class_data_dir: str = "",
88:c3_class_guidance_scale: float = 7.5,
89:c3_class_infer_steps: int = 60,
90:c3_class_negative_prompt: str = "",
91:c3_class_prompt: str = "",
92:c3_class_token: str = "",
93:c3_instance_data_dir: str = "",
94:c3_instance_prompt: str = "",
95:c3_instance_token: str = "",
96:c3_max_steps: int = -1,
97:c3_n_save_sample: int = 1,
98:c3_num_class_images: int = 0,
99:c3_sample_seed: int = -1,
100:c3_save_guidance_scale: float = 7.5,
101:c3_save_infer_steps: int = 60,
102:c3_save_sample_negative_prompt: str = "",
103:c3_save_sample_prompt: str = "",
104:c3_save_sample_template: str = "",
105:concepts_list=None
"""

'\n1:model_name: str = "",\n2:adam_beta1: float = 0.9,\n3:adam_beta2: float = 0.999,\n4:adam_epsilon: float = 1e-8,\n5:adam_weight_decay: float = 0.01,\n6:attention: str = "default",\n7:center_crop: bool = True,\n8:concepts_path: str = "",\n9:custom_model_name: str = "",\n10:epoch_pause_frequency: int = 0,\n11:epoch_pause_time: int = 0,\n12:gradient_accumulation_steps: int = 1,\n13:gradient_checkpointing: bool = True,\n14:half_model: bool = False,\n15:has_ema: bool = False,\n16:hflip: bool = False,\n17:learning_rate: float = 0.00000172,\n18:lora_learning_rate: float = 1e-4,\n19:lora_txt_learning_rate: float = 5e-5,\n20:lr_scheduler: str = \'constant\',\n21:lr_warmup_steps: int = 0,\n22:max_token_length: int = 75,\n23:max_train_steps: int = 1000,\n24:mixed_precision: str = "fp16",\n25:model_path: str = "",\n26:not_cache_latents=False,\n27:num_train_epochs: int = 1,\n28:pad_tokens: bool = True,\n29:pretrained_vae_name_or_path: str = "",\n30:prior_loss_weight: float = 1.0,\n31:resolution:

In [5]:
dreambooth_params = [
  "",
  0.9,
  0.999,
  1e-08,
  0.01,
  "default",
  False,
  "",
  "",
  0.0,
  60.0,
  1,
  True,
  False,
  "",
  True,
  2e-06,
  0.0002,
  0.0002,
  "constant",
  500,
  75,
  0,
  "no",
  "",
  True,
  100,
  True,
  "",
  1,
  512,
  "",
  1,
  True,
  500,
  500,
  True,
  False,
  False,
  "",
  "",
  False,
  1,
  True,
  False,
  False,
  False,
  False,
  False,
  "",
  "/opt/ml/input/data/images/",
  7.5,
  40,
  "",
  "a photo of Erwin Rommel",
  "",
  "/opt/ml/input/data/images/",
  "Erwin Rommel",
  "",
  -1,
  1,
  0,
  -1,
  7.5,
  40,
  "",
  "",
  "",
  "",
  7.5,
  40,
  "",
  "",
  "",
  "",
  "",
  "",
  -1,
  1,
  0,
  -1,
  7.5,
  40,
  "",
  "",
  "",
  "",
  7.5,
  40,
  "",
  "",
  "",
  "",
  "",
  "",
  -1,
  1,
  0,
  -1,
  7.5,
  40,
  "",
  "",
  ""
]

In [6]:
import uuid, json
dreambooth_config_id = "dreambooth_train_config"
dreambooth_config_file =f'{dreambooth_config_id}.json'
json.dump(dreambooth_params, open(dreambooth_config_file,'w'), indent=6)

In [7]:
print(json.dumps(dreambooth_params))

["", 0.9, 0.999, 1e-08, 0.01, "default", false, "", "", 0.0, 60.0, 1, true, false, "", true, 2e-06, 0.0002, 0.0002, "constant", 500, 75, 0, "no", "", true, 100, true, "", 1, 512, "", 1, true, 500, 500, true, false, false, "", "", false, 1, true, false, false, false, false, false, "", "/opt/ml/input/data/images/", 7.5, 40, "", "a photo of Erwin Rommel", "", "/opt/ml/input/data/images/", "Erwin Rommel", "", -1, 1, 0, -1, 7.5, 40, "", "", "", "", 7.5, 40, "", "", "", "", "", "", -1, 1, 0, -1, 7.5, 40, "", "", "", "", 7.5, 40, "", "", "", "", "", "", -1, 1, 0, -1, 7.5, 40, "", "", ""]


In [71]:
#image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/all-in-one-ai-stable-diffusion-webui-training'.format(account_id, region_name)
image_uri = '687912291502.dkr.ecr.ap-southeast-1.amazonaws.com/dreambooth-finetuning-v3:latest'
models_s3uri = 's3://{0}/stable-diffusion/models/'.format(bucket)
dreambooth_s3uri = 's3://{0}/stable-diffusion/dreambooth/'.format(bucket)
dreambooth_config_s3uri = 's3://{0}/stable-diffusioni/dreambooth-config/'.format(bucket)

In [10]:
print(dreambooth_config_file)
print(dreambooth_config_s3uri)
!aws s3 cp $dreambooth_config_file $dreambooth_config_s3uri

dreambooth_train_config.json
s3://sagemaker-ap-southeast-1-687912291502/stable-diffusioni/dreambooth-config/
upload: ./dreambooth_train_config.json to s3://sagemaker-ap-southeast-1-687912291502/stable-diffusioni/dreambooth-config/dreambooth_train_config.json


In [11]:
def json_encode_hyperparameters(hyperparameters):
    for (k, v) in hyperparameters.items():
        print(k, v)
    
    return {k: json.dumps(v) for (k, v) in hyperparameters.items()}

train_args = {
    'train_dreambooth_settings': {
        'db_create_new_db_model': True, 
        'db_new_model_name': 'aws-db-new-model', 
        'db_new_model_src': '768-v-ema.ckpt', 
        'db_new_model_scheduler': 'ddim', 
        'db_create_from_hub': False, 
        'db_new_model_url': '', 
        'db_new_model_token': '', 
        'db_new_model_extract_ema': False, 
        'db_model_name': '', 
        'db_lora_model_name': '', 
        'db_lora_weight': 1, 
        'db_lora_txt_weight': 1, 
        'db_train_imagic_only': False, 
        'db_use_subdir': False, 
        'db_custom_model_name': '', 
        'db_train_wizard_person': False, 
        'db_train_wizard_object': True, 
        'db_performance_wizard': True
    }
}


hyperparameters = {
    'train-args': json.dumps(train_args),
    'train-task': 'dreambooth',
    'sd-models-s3uri': models_s3uri,
    'db-models-s3uri': dreambooth_s3uri,
    'ckpt': '/opt/ml/input/data/models/768-v-ema.ckpt',
    'dreambooth-config-id': dreambooth_config_id,
    'api-endpoint': 'noapi'
}

hyperparameters = json_encode_hyperparameters(hyperparameters)



train-args {"train_dreambooth_settings": {"db_create_new_db_model": true, "db_new_model_name": "aws-db-new-model", "db_new_model_src": "768-v-ema.ckpt", "db_new_model_scheduler": "ddim", "db_create_from_hub": false, "db_new_model_url": "", "db_new_model_token": "", "db_new_model_extract_ema": false, "db_model_name": "", "db_lora_model_name": "", "db_lora_weight": 1, "db_lora_txt_weight": 1, "db_train_imagic_only": false, "db_use_subdir": false, "db_custom_model_name": "", "db_train_wizard_person": false, "db_train_wizard_object": true, "db_performance_wizard": true}}
train-task dreambooth
sd-models-s3uri s3://sagemaker-ap-southeast-1-687912291502/stable-diffusion/models/
db-models-s3uri s3://sagemaker-ap-southeast-1-687912291502/stable-diffusion/dreambooth/
ckpt /opt/ml/input/data/models/768-v-ema.ckpt
dreambooth-config-id dreambooth_train_config
api-endpoint noapi


In [12]:
print(models_s3uri)

s3://sagemaker-ap-southeast-1-687912291502/stable-diffusion/models/


#################与webui绑定的training BYOC#########################

In [ ]:
#!aws s3 cp ./768-v-ema.ckpt   $models_s3uri
#!aws s3 cp ./768-v-ema.yaml   $models_s3uri
image_uri = '687912291502.dkr.ecr.ap-southeast-1.amazonaws.com/dreambooth-finetuning-v3-with-webui:latest'
#image_uri = '687912291502.dkr.ecr.ap-southeast-1.amazonaws.com/dreambooth-finetuning-v3:latest'
#image_uri = '687912291502.dkr.ecr.ap-southeast-1.amazonaws.com/dreambooth-finetuning-v3-withw-webui:latest'
instance_type = 'ml.g4dn.2xlarge'
#image_uri = '687912291502.dkr.ecr.ap-southeast-1.amazonaws.com/dreambooth-finetuning-v3:latest'
from sagemaker.estimator import Estimator
inputs = {
    'images': images_s3uri,
    'models': models_s3uri,
    'config': dreambooth_config_s3uri
}
# Please exectute tools/prepare.py with the path to your model files directory.
# Make sure the 768-v-ema.ckpt and 768-v-ema.yaml have been uploaded to sd_models_s3uri
estimator = Estimator(
    role = role,
    instance_count=1,
    instance_type = instance_type,
    image_uri = image_uri,
    hyperparameters = hyperparameters
)
estimator.fit(inputs)

#################不与webui绑定的training BYOC#########################

In [63]:
!aws s3 ls s3://sagemaker-ap-southeast-1-687912291502/dreambooth/trained_models/
#!aws s3 rm "s3://sagemaker-ap-southeast-1-687912291502/dreambooth/trained_models/*"

                           PRE .ipynb_checkpoints/
                           PRE dir1/
                           PRE test_dir/
2023-02-20 09:03:03       3374 helper.py
2023-02-20 08:57:04        537 model_index.json
2023-02-20 09:03:03       1546 prepare.py


In [75]:
import json
def json_encode_hyperparameters(hyperparameters):
    for (k, v) in hyperparameters.items():
        print(k, v)
    
    return {k: json.dumps(v) for (k, v) in hyperparameters.items()}



image_uri = '687912291502.dkr.ecr.ap-southeast-1.amazonaws.com/dreambooth-finetuning-v3:latest'
instance_type = 'ml.g4dn.4xlarge'

instance_prompt="Erwin\ Rommel"
prior_preservation_class_prompt="a\ photo\ of\ Erwin\ Rommel"
s3_model_output_location='s3://{}/{}/{}'.format(bucket, 'dreambooth', 'trained_models')
#model_name="CompVis/stable-diffusion-v1-4"
model_name="runwayml/stable-diffusion-v1-5"
#model_name="ClueAI/ChatYuan-large-v1"
#model_name="stabilityai/stable-diffusion-2"
instance_dir="/opt/ml/input/data/images/"
class_dir="/opt/ml/input/data/images/"



environment = {
    'PYTORCH_CUDA_ALLOC_CONF':'max_split_size_mb:32',
    'LD_LIBRARY_PATH':"${LD_LIBRARY_PATH}:/opt/conda/lib/"
}

hyperparameters = {
                    'model_name':'aws-trained-dreambooth-model',
                    'mixed_precision':'fp16',
                    'pretrained_model_name_or_path': model_name, 
                    'instance_data_dir':instance_dir,
                    'class_data_dir':class_dir,
                    'with_prior_preservation':True,
                    #'models_path': '/opt/ml/model/',
                    'models_path': '/opt/ml/output/',
                    'manul_upload_model_path':s3_model_output_location,
                    'instance_prompt': instance_prompt, 
                    'class_prompt':prior_preservation_class_prompt,
                    'resolution':512,
                    'train_batch_size':1,
                    'sample_batch_size': 1,
                    'gradient_accumulation_steps':1,
                    'learning_rate':2e-06,
                    'lr_scheduler':'constant',
                    'lr_warmup_steps':100,
                    'num_class_images':0,
                    'max_train_steps':800,
                    'save_steps':800,
                    #'attention':'flash_attention',
                    'attention':'xformers',
                    'prior_loss_weight': 0.5,
                    'use_ema':True,
                    'train_text_encoder':False,
                    'not_cache_latents':True,
                    'gradient_checkpointing':True,
                    'save_use_epochs': False,
                    'use_8bit_adam': False
}

hyperparameters = json_encode_hyperparameters(hyperparameters)

from sagemaker.estimator import Estimator
inputs = {
    'images': images_s3uri
}

estimator = Estimator(
    role = role,
    instance_count=1,
    instance_type = instance_type,
    image_uri = image_uri,
    hyperparameters = hyperparameters,
    environment = environment
)
estimator.fit(inputs)

model_name aws-trained-dreambooth-model
mixed_precision fp16
pretrained_model_name_or_path runwayml/stable-diffusion-v1-5
instance_data_dir /opt/ml/input/data/images/
class_data_dir /opt/ml/input/data/images/
with_prior_preservation True
models_path /opt/ml/output/
manul_upload_model_path s3://sagemaker-ap-southeast-1-687912291502/dreambooth/trained_models
instance_prompt Erwin\ Rommel
class_prompt a\ photo\ of\ Erwin\ Rommel
resolution 512
train_batch_size 1
sample_batch_size 1
gradient_accumulation_steps 1
learning_rate 2e-06
lr_scheduler constant
lr_warmup_steps 100
num_class_images 0
max_train_steps 800
save_steps 800
attention xformers
prior_loss_weight 0.5
use_ema True
train_text_encoder False
not_cache_latents True
gradient_checkpointing True
save_use_epochs False
use_8bit_adam False
2023-02-27 10:36:15 Starting - Starting the training job...
2023-02-27 10:36:38 Starting - Preparing the instances for trainingProfilerReport-1677494174: InProgress
......
2023-02-27 10:37:41 Downlo

UnexpectedStatusException: Error for Training job dreambooth-finetuning-v3-2023-02-27-10-36-14-292: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage ""
Command "/opt/conda/bin/python train.py --attention xformers --class_data_dir /opt/ml/input/data/images/ --class_prompt a\ photo\ of\ Erwin\ Rommel --gradient_accumulation_steps 1 --gradient_checkpointing True --instance_data_dir /opt/ml/input/data/images/ --instance_prompt Erwin\ Rommel --learning_rate 2e-06 --lr_scheduler constant --lr_warmup_steps 100 --manul_upload_model_path s3://sagemaker-ap-southeast-1-687912291502/dreambooth/trained_models --max_train_steps 800 --mixed_precision fp16 --model_name aws-trained-dreambooth-model --models_path /opt/ml/output/ --not_cache_latents True --num_class_images 0 --pretrained_model_name_or_path runwayml/stable-diffusion-v1-5 --prior_loss_weight 0.5 --resolution 512 --sample_batch_size 1 --save_steps 800 --save_use_epochs False --train_batch_size 1 --train_text_e

In [23]:
dreambooth_model_data = estimator.model_data
print("Model artifact saved at:\n", dreambooth_model_data)

Model artifact saved at:
 s3://sagemaker-ap-southeast-1-687912291502/dreambooth-finetuning-v3-2023-02-20-03-54-22-434/output/model.tar.gz


In [66]:
!aws s3 ls s3://sagemaker-ap-southeast-1-687912291502/dreambooth/trained_models/

2023-02-20 09:56:38        601 config.json
2023-02-20 09:56:37  167402961 diffusion_pytorch_model.bin
